# Counterfactual explanation for Mobile Price Classification dataset

TODO: explain the task and something about counterfactual explanation.

## Installation and import of the needed libraries

First of all we clone the Github repository in order to have all the needed files to perform our analysis.

In [1]:
!mkdir my_repo
!git clone https://github.com/Valendrew/counterfactual-explanations.git my_repo

Cloning into 'my_repo'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 30 (delta 10), reused 27 (delta 7), pack-reused 0
Unpacking objects: 100% (30/30), 6.23 KiB | 910.00 KiB/s, done.


- OMLT, version 1.1
- ONNX, version 1.12.0. A newer version is available (1.13.1) but it gives conflict error with TensorFlow. If the latter isn't needed, then the installed version can be upgraded.

Then we can install all the libraries from the ```requirements.txt``` file.

In [2]:
!pip install -r my_repo/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 5.1 MB/s eta 0:00:00


In [6]:
# python modules
import configparser
import math

# 3rd party modules
import gdown
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# user modules
from my_repo.utils import util_data, util_plot, util_network

In order to manage the randomness we wrote the following cell to make the experiments reproducible, check the following [link](https://scikit-learn.org/stable/common_pitfalls.html#controlling-randomness).

In [4]:
def set_default_randomness(seed: int):
    rng = np.random.RandomState(seed)
    return rng

SEED = 42
rng = set_default_randomness(SEED)

Moreover we import the configuration file to have all the parameters along the notebook.

In [5]:
config = configparser.ConfigParser()
config.read("my_repo/config.ini")

['my_repo/config.ini']

## Dataset exploration

For the project we use the following dataset from Kaggle: [dataset](https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification).

It gives information about different characteristics of the smartphones (e.g. RAM, cores, ...) and the target variable is the price range of the device.

In [ ]:
ds_conf = config["DATASET"]
# Download the training and test datasets
util_data.download_helper(ds_conf["url_train"], ds_conf["name_train"], quiet=False)
util_data.download_helper(ds_conf["url_test"], ds_conf["name_test"], quiet=False)

After that we downloaded the two datasets we can load them as a dataframe to leverage Pandas built-in functions.

In [ ]:
df_train = util_data.read_csv(ds_conf["name_train"])
df_train = util_data.read_csv(ds_conf["name_test"])